In [17]:
import os
import cv2
import random
import numpy as np

images = []

with open("./words.txt","r",encoding="utf-8") as file:
    for line in file:
        if line[0] != "#":
            data = {}
            data['attributes'] = line.split(" ")
            data['imagePath'] = data.get("attributes")[0].split("-")
            data['checkWord'] = data.get("attributes")[1]
            data['thresh'] = int(data.get("attributes")[2])
            data['x'] = int(data.get("attributes")[3])
            data['y'] = int(data.get("attributes")[4])
            data['w'] = int(data.get("attributes")[5])
            data['h'] = int(data.get("attributes")[6])
            data['word'] = data.get("attributes")[8]

            outerFolder = data.get("imagePath")[0]
            innerFolder = data.get("imagePath")[0] + "-" + data.get("imagePath")[1]
            file = data.get("attributes")[0]
            pathofImage = "./" + outerFolder + "/" + innerFolder + "/" + file + ".png"
            data['imagePath'] = pathofImage
            bad_samples = ['a01-117-05-02', 'r06-022-03-05']

            if data.get("checkWord") == "err" or data.get("attributes")[0] in bad_samples:
                continue

            if data.get("w") < 20 or data.get("h") < 20:
                continue


            images.append(data)



In [28]:

for ob in images:
    image = cv2.imread(ob.get("imagePath"))
    
    if ob.get("w") > 128 and ob.get("h") > 32:
        resizedImage = cv2.resize(image, (128, 32), interpolation=cv2.INTER_AREA)
        
    elif ob.get("w") < 128 and ob.get("h") < 32:
        if 128 - ob.get("w") < 32 - ob.get("h"):
            resizedImage = cv2.resize(image, (128,image.shape[0]), interpolation=cv2.INTER_AREA)
        elif 128 - ob.get("w") > 32 - ob.get("h"):
            resizedImage = cv2.resize(image, (image.shape[1],32), interpolation=cv2.INTER_AREA)
            
    elif ob.get("w") < 128 and ob.get("h") > 32:
            resizedImage = cv2.resize(image, (image.shape[1],32), interpolation=cv2.INTER_AREA)
        
    elif ob.get("w") > 128 and ob.get("h") < 32:
            resizedImage = cv2.resize(image, (128, image.shape[0]), interpolation=cv2.INTER_AREA)
            
    
    
    greyImage = cv2.cvtColor(resizedImage,cv2.COLOR_BGR2GRAY)
    
    
    threshold , binarizedImage = cv2.threshold(greyImage , ob.get("thresh") , 255 , cv2.THRESH_BINARY)
    
    
    emptyImage = np.ones((32,128),np.uint8)*255
    
    if binarizedImage.shape[0] == 32:
        start = random.randint(0,128-binarizedImage.shape[1])
        end = start + binarizedImage.shape[1]
        emptyImage[:,start:end] = binarizedImage
        
    if binarizedImage.shape[1] == 128:    
        start = random.randint(0,32-binarizedImage.shape[0])
        end = start + binarizedImage.shape[0]
        emptyImage[start:end,:] = binarizedImage
    
    ob["image"] = emptyImage
    


In [29]:
for i in range(10):
    cv2.imwrite('test' + str(i) +'.png',images[i].get("image"))